In [21]:
import os
import glob
import pickle as pkl
import datetime

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split, cross_validate
from sklearn.metrics import r2_score, \
    explained_variance_score, normalized_mutual_info_score, mutual_info_score, make_scorer, mean_absolute_error
from sklearn.model_selection import KFold
from sklearn.utils import shuffle

from skll.metrics import spearman, pearson

from keras.losses import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from keras import backend as K
from keras import regularizers
from keras import optimizers
from keras.callbacks import EarlyStopping

from hyperopt import Trials, fmin, tpe, hp, STATUS_OK

from mlxtend.preprocessing import shuffle_arrays_unison

from pylab import rcParams
rcParams['figure.figsize'] = 8, 8

Custom model implementations and functions are stored in `src/dairyml.py`

In [2]:
from dairyml import PerfectClassifierMeanRegressor, plot_r2, plot_coefficients

## Import the Data
Load the data from the pickle files created in `preproccess.ipynb`

In [3]:
with open("../pkl/data/data_outliers_removed", "rb" ) as f:
    [X, Y] = pkl.load(f)

## Modelling with Feed-Forward Neural Network (FFNN)

Currently using 5 folds to save time

In [4]:
splitter= KFold(n_splits=5,shuffle=True,random_state=7)

#### define r^2 metric for keras model

In [5]:
def r2_keras(y_true, y_pred):
    SS_res =  K.sum(K.square( y_true-y_pred ))
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) )
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

### Function to build model

In [6]:
def create_model(hidden_layers,num_nodes,alpha=0.01,lr=.001): 
    
    reg = regularizers.l2(alpha)
    
    model = Sequential()
    
    model.add(Dense(num_nodes, input_dim=X.shape[1], activation='relu', kernel_regularizer = reg))
    
    for i in range(0,hidden_layers-1):
        model.add(Dense(num_nodes, activation='relu', kernel_regularizer = reg))
        
    model.add(Dense(1, activation='linear'))
    
    adam = optimizers.Adam(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    
    #add r2 and other metrics
    model.compile(loss='mean_squared_error', optimizer=adam,metrics=['mean_absolute_error',r2_keras])
    return model

### Use r2 scoring in cv

In [57]:
scoring = {'r2':make_scorer(r2_score), 
           'SRC':make_scorer(spearman), 
           'PCC':make_scorer(pearson), 
           'MI':make_scorer(mutual_info_score), 
           'MAE':make_scorer(mean_absolute_error)}

In [464]:
try:
    overall_results = pd.read_csv('../reports/model_results.csv',index_col=0)
except FileNotFoundError:
    overall_results = pd.DataFrame(columns = scoring.keys())

### Plot learning curves for train test split to estimate appropriate number of epochs
Can replace this with just early stopping

### Define objective function (1-r2) for hyperopt

need to print more information here:
  - early stopping status for each split of cv
  - scores for each part of cv

In [112]:
# EPOCHS = 2000
# X_shuf, Y_shuf = shuffle(X,Y,random_state=7)

def objective(params):
    hidden_layers = int(params['hidden_layers'])
    num_nodes = int(params['num_nodes'])
    alpha = params['alpha']
    lr = params['lr']
    epochs = int(params['epochs'])
    
    # Print configuration
    print('hidden_layers: {}'.format(hidden_layers))
    print('num_nodes: {}'.format(num_nodes))
    print('alpha: {}'.format(alpha))
    print('lr: {}'.format(lr))
    print('epochs: {}'.format(epochs))
    
    # build keras model with given configuration
    model = KerasRegressor(build_fn=create_model,
                           hidden_layers=hidden_layers,
                           num_nodes=num_nodes,
                           alpha=alpha,
                           lr=lr,
                           epochs=epochs, 
                           verbose=0)
    # get cv results
#     early_stopping = EarlyStopping(monitor='val_loss',min_delta=0,patience=20,mode='min',verbose=1)
    results = cross_validate(model,X,Y,cv=splitter,scoring=scoring)
#     history = model.fit(X_shuf,Y_shuf,validation_split=0.2,callbacks=[early_stopping],epochs = EPOCHS)
    
#     print('early stopping: {}'.format(early_stopping.stopped_epoch))
    # get average r2 from cv
    r2 = np.mean(results['test_r2'])
    
#     r2 = history.history['val_r2_keras'][-1]
    
    # convert r2 to a loss
    loss = 1 - r2
    
    # print r2 result
    print('R^2: {}'.format(r2))
    print('\n')
    
    # return loss
    return {'loss': loss, 'params': params, 'r2': r2, 'cv_results': results, 'status': STATUS_OK}
#     return {'loss': loss, 'params': params, 'early_stopping': early_stopping.stopped_epoch, 'status': STATUS_OK}

### Define the parameter space

min, max, stepsize

In [113]:
space = {
            'hidden_layers': hp.quniform('hidden_layers', 1, 2, 1),
            'num_nodes': hp.qloguniform('num_nodes', np.log(3), np.log(100), 1),
            'lr': hp.loguniform('lr', np.log(1e-4), np.log(1e-2)),
            'alpha': hp.loguniform('alpha', np.log(1e-4), np.log(1e-1)),
            'epochs': hp.qnormal('epochs',150,25,5)
        }

### Trials object to store results

In [100]:
trials = Trials()

### Run hyperopt

### need to print early stopping for each iter of cross_validate

In [111]:
# MAX_EVALS = 100
# # Optimize
# best = fmin(fn = objective, space = space, algo = tpe.suggest, max_evals = MAX_EVALS, trials = trials)

In [117]:
def run_trials():

    trials_step = 1  # how many additional trials to do after loading saved trials. 1 = save after iteration
    max_trials = 3  # initial max_trials. put something small to not have to wait
    
    try:  # try to load an already saved trials object, and increase the max
        trials = load_trials()
#         trials = Trials()
        max_trials = len(trials.trials) + trials_step
        print("Rerunning from {} trials to {} (+{}) trials".format(len(trials.trials), max_trials, trials_step))
    except:  # create a new trials object and start searching
        trials = Trials()

    best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=max_trials, trials=trials)

    print("Best:", best)
    
    # save the trials object
    save_trials(trials)


In [ ]:
while True:
    run_trials()

In [92]:
epochs=2000

def create_from_params(params):
    hidden_layers = int(params['hidden_layers'])
    num_nodes = int(params['num_nodes'])
    alpha = params['alpha']
    lr = params['lr']
    
    model = KerasRegressor(build_fn=create_model,
                       hidden_layers=hidden_layers,
                       num_nodes=num_nodes,
                       alpha=alpha,
                       lr=lr,
                       epochs=120, 
                       verbose=0)
    return model
    

In [93]:
model=create_from_params(best)

results = cross_validate(model,X,Y,cv=splitter,scoring=scoring)

In [95]:
# np.mean(results['test_r2'])

0.584380188790888

In [27]:
def save_params(params):
    save_string = 'FFNN_best_params-' + str(datetime.datetime.now().strftime("%Y-%m-%d %H-%M %p"))
    params_dir = "../pkl/params/FFNN/"
    
    try:
        os.makedirs(params_dir)
    except FileExistsError:
        pass
    
    with open(params_dir + save_string, "wb" ) as f:
        f.seek(0)
        pkl.dump(params,f)

def load_params():
    params_dir = "../pkl/params/FFNN/*"
    list_of_files = glob.glob(params_dir)
    latest_file = max(list_of_files, key=os.path.getctime)
    print('loading {}'.format(latest_file))
    
    with open(latest_file, "rb") as f:
        f.seek(0)
        params = pkl.load(f)
        
    return params

In [28]:
def save_trials(trials):
    save_string = 'FFNN_trials-' + str(datetime.datetime.now().strftime("%Y-%m-%d %H-%M %p"))
    trials_dir = '../pkl/trials/FFNN/'
    
    try:
        os.makedirs(trials_dir)
    except FileExistsError:
        pass
    
    with open(trials_dir + save_string, "wb" ) as f:
        f.seek(0)
        pkl.dump(trials,f)
        
    print('saved to {}'.format(trials_dir + save_string))

def load_trials():
    trials_dir = "../pkl/trials/FFNN/*"
    list_of_files = glob.glob(trials_dir)
    latest_file = max(list_of_files, key=os.path.getctime)
    print('loading {}'.format(latest_file))
    
    with open(latest_file, "rb") as f:
        f.seek(0)
        trials = pkl.load(f)
        
    return trials

In [444]:
save_params(best)
load_params()

loading ../pkl/params/FFNN\FFNN_best_params-2019-01-21 12-43 PM


{'alpha': 0.005660851233300683,
 'hidden_layers': 1.0,
 'lr': 0.001994170783323986,
 'num_nodes': 59.0}

In [29]:
save_trials(trials)
# load_trials()

saved to ../pkl/trials/FFNN/FFNN_trials-2019-01-24 04-02 AM


Trials so far were conducted with only r2 in the scoring dict, so I added the other metrics then re-ran with what was supposedly the best configuration. This achieved .71 r^2 above, but only .65 below. This is not very consistent at all, maybe need to increase the number of folds in CV, or increase number of epochs. Also thinking about setting number of epochs much higher and just using early stopping to get to the appropriate number. 

In [456]:
# final_model_results = objective(trials.best_trial['result']['params'])

hidden_layers: 1
num_nodes: 59
alpha: 0.005660851233300683
lr: 0.001994170783323986
R^2: 0.655077183043743




In [466]:
for score_name in scoring.keys():
    overall_results.loc['FFNN',score_name] = np.round(np.mean(final_model_results['cv_results']['test_'+score_name]),2)
overall_results

,r2,SRC,PCC,MI,MAE
Dummy Mean,-0.02,0.00,-0.00,-0.00,1.94
Dummy Median All,-0.32,0.00,-0.00,-0.00,1.68
Dummy Median Nonzero,-0.08,0.00,-0.00,-0.00,1.77
"Perfect Clasif., Mean Regr.",0.13,0.73,0.41,0.53,1.53
Lasso,0.45,0.61,0.70,3.07,1.23
Bounded Lasso,0.55,0.64,0.75,2.87,1.08
Bounded Lasso + LogReg,0.64,0.80,0.82,2.66,0.86
FFNN,0.66,0.70,0.83,3.58,0.96


In [467]:
overall_results.to_csv('../reports/model_results.csv')